In [ ]:
import chess
import chess.engine
import chess.pgn
from stockfish import Stockfish
import numpy as np
from collections import Counter
import uuid
import copy
import os
import random

#PARAMETERS TO ADJUST
game_agents = 'experiment'
think_time = 4000
max_non_mistakes = 3
suboptimal_move_tolerance = 300
opening_plys = 20


In [ ]:
#Prepare array of Engine names using txt file I've set up
def get_engine_name_array():
    engine_names = []
    with open(os.path.join(os.getcwd()  + "/list_of_engine_names.txt")) as f:
        lines = f.readlines()
        for l in lines:
            #Transform into CamelCase
            l = l.strip()
            engine_names.append(l)
            l = "".join([x.capitalize() for x in l.split()])
            engine_names.append(l)
    return engine_names

player_turn = {0: 'White', 1: 'Black'}
sf = Stockfish(path="./stockfish_15/stockfish_15.exe",
                depth=10,
                parameters={"Threads": 4, "Minimum Thinking Time": think_time})

def getCentipawn(el):
    return el.get('Centipawn')

def getMate(el):
    return el.get('Mate')

def get_piece_count(board):
    x = 0
    for i in range(2,6):
        x = x + len(board.pieces(piece_type=i,color=True))
        x = x + len(board.pieces(piece_type=i,color=False))
    return x

def process_game(game, engine_colour=""):
    
    game_rescan = copy.copy(game)
    board = game.board()
 
    #1 - RECORD POSITIONAL OR TACTICAL BOARD POSITIONS

    p_or_t_reference = []
    for i, move in enumerate(game.mainline_moves()):
        if i < opening_plys:
            board.push(move)
            continue

        sf.set_fen_position(board.fen())

        if get_piece_count(board) <= 4:
            #print("Entered Endgame (4 pieces left)")
            break

        best_moves = sf.get_top_moves(5)
        best = getCentipawn(best_moves[0])
        good_moves = 0
        for m in best_moves:
            if getMate(m) or ((best - getCentipawn(m)) > suboptimal_move_tolerance):
                break
            else:
                good_moves+=1
        if good_moves <= max_non_mistakes:
            #tactical
            p_or_t_reference.append('t')
            #print("Ply={i}, Colour={c}, Tactical, GoodMoves={n}: ".format(i=i+1,c=player_turn[i%2],n=good_moves), board.fen())

        else:
            #postional
            p_or_t_reference.append('p')
            #print("Ply={i}, Colour={c}, Positional: ".format(i=i+1,c=player_turn[i%2]), board.fen())

        board.push(move)
    
    #2 - GET ANY KEY SEQUENCES

    key_sequence_starting_plys = []
    SEQUENCE_LENGTH = 20
    POSITIONAL_REQUIREMENT = 8 

    for i in range(len(p_or_t_reference)-SEQUENCE_LENGTH):
        if (p_or_t_reference[i] == 't'):
            continue
        counts = dict(Counter(p_or_t_reference[i:i+10]))
        if(counts['p'] >= POSITIONAL_REQUIREMENT):
            key_sequence_starting_plys.append(i+opening_plys)

    #3 - REMOVE OVERLAPPING STARTING POINTS

    temp = np.array(key_sequence_starting_plys)
    no_overlap_key_plys = []
    while len(temp):
        no_overlap_key_plys.append(temp[0])
        temp = temp[temp >= (temp[0]+SEQUENCE_LENGTH)]

    #4 - SAVE KEY SEQUENCE FENS

    fens_to_save = np.empty((len(no_overlap_key_plys),20), dtype=object)

    board_rescan = game_rescan.board()

    move_num = 0
    sequence_num = 0

    for i, move in enumerate(game.mainline_moves()):
        if sequence_num == len(no_overlap_key_plys):
            break
        if i < no_overlap_key_plys[sequence_num]:
            board_rescan.push(move)
            continue
        elif i < no_overlap_key_plys[sequence_num] + 20:
            fens_to_save[sequence_num][move_num] = board_rescan.fen()
            move_num = move_num + 1
            board_rescan.push(move)
            if move_num == 20:
                move_num = 0
                sequence_num = sequence_num + 1

    for f_ctr in range(len(fens_to_save)):
        f = open("./train/"+game_agents+"/"+ engine_colour + "_" + str(uuid.uuid4())+".txt",'w')
        for el in fens_to_save[f_ctr]:
            f.write(str(el)+'\n')
        f.close()

In [ ]:
def process_game_random(game, engine_colour=""):
    
    board = game.board()
    fens_to_save = np.empty((1,20), dtype=object)
    
    #need to get the move count 
    ply_count = len([g for g in game.mainline_moves()])

    try:
        starting_ply = random.randrange(20, ply_count - 20)
    except:
        return

    move_num = 0

    for i, move in enumerate(game.mainline_moves()):
        if i < starting_ply:
            board.push(move)
            continue
        elif i < (starting_ply + 20):
            fens_to_save[0][move_num] = board.fen()
            move_num = move_num + 1
            board.push(move)
        else:
            break

    for f_ctr in range(len(fens_to_save)):
        f = open("./random/train/"+game_agents+"/"+ engine_colour + "_" + str(uuid.uuid4())+".txt",'w')
        for el in fens_to_save[f_ctr]:
            f.write(str(el)+'\n')
        f.close()
    

In [ ]:

engine_names = get_engine_name_array()
t=1
for filename in os.listdir(os.getcwd() + "/pgns/experiment/"):
   pgn = open(os.path.join(os.getcwd()  + "/pgns/experiment/", filename))
   for i in range(5):
      try:
         #will eventually run out of games in pgn file
         game = chess.pgn.read_game(pgn)
         #headers = chess.pgn.read_headers(pgn)
         #w = headers.get("White","")
         #b = headers.get("Black","")
         process_game(game)
   
         #Below was used for AvH / HvA games specifically
         '''
         if w in engine_names and b in engine_names:
            continue
         elif w in engine_names:
            process_game_random(game, engine_colour="w")
         elif b in engine_names:
            process_game_random(game, engine_colour="b")
         '''
      except Exception as e:
         print(e)
         pgn.close()
         break
   
      

In [ ]:
#EXAMPLE ANALYSIS

b = chess.Board("rn4k1/pb3p1p/1p4p1/1N2p3/4P3/3rB3/P4PPP/1R3RK1 w - - 0 18")
display(b)
s = Stockfish(path="./stockfish_15/stockfish_15.exe",
                depth=10,
                parameters={"Threads": 4, "Minimum Thinking Time": think_time})
s.set_fen_position(b.fen())
m = s.get_top_moves(5)
print(m)